# SeaSoar Processing

1. Load data
2. Identify upcasts and downcasts
3. Flag spurious pressure/data points by moving median filter
4. Bin sensor data to regular pressures
5. Flag outliers in signals

In [1]:
source('source.r')

Packages Loaded:
ncdf4 R.matlab openxlsx RColorBrewer compiler lattice geosphere readxl data.table rworldmap rworldxtra


Loading required package: ncdf4
Loading required package: R.matlab
R.matlab v3.6.1 (2016-10-19) successfully loaded. See ?R.matlab for help.

Attaching package: ‘R.matlab’

The following objects are masked from ‘package:base’:

    getOption, isOpen

Loading required package: openxlsx
Loading required package: RColorBrewer
Loading required package: compiler
Loading required package: lattice
Loading required package: geosphere
Loading required package: readxl
Loading required package: data.table
Loading required package: rworldmap
Loading required package: sp
Error: package or namespace load failed for ‘rworldmap’ in loadNamespace(j <- i[[1L]], c(lib.loc, .libPaths()), versionCheck = vI[[j]]):
 there is no package called ‘fields’
Loading required package: rworldxtra


In [4]:
input.dir = '../../Models/SeaSoar/SeaSoar2/'

files = list.files(input.dir, full.names = TRUE)
files = files[grepl('.mat', files)]
files = files[!grepl('.rdata', files)]
length(files)

[1] 46

In [6]:
str(files)
files[1]

 chr [1:46] "../../Models/SeaSoar/SeaSoar2//FLT_DAT001.mat" ...


[1] "../../Models/SeaSoar/SeaSoar2//FLT_DAT001.mat"

In [ ]:
get.data = function(input.dir, file) {
    data = readMat(paste0(input.dir, file), fixNames = TRUE)
    names = unlist(dimnames(data$data)[1])

    ## Partition up datea
    eng = as.data.frame(data$data[2:12])
    sensor = as.data.frame(data$data[13:36])
    colnames(sensor) = names[13:36]
    colnames(eng) = names[2:12]
    
    sensor$time = seq(from = eng$time[1], by = 1/24, length.out = nrow(sensor))
    
    return(list(eng = eng, sensor = sensor))
}

#### Flag Outliers in pressure
flag.p.outliers = function(data, dp = 0.5, verbose = TRUE, window = 25) {
    l = which(abs(data$sensor$p - runmed(data$sensor$p, window)) > dp)
    
    data$sensor$p.FLAG = 1 # keep
    data$sensor$p.FLAG[l] = 3 # reject
    
    if (verbose) {
        n = length(l)
        perc = floor(length(l) / nrow(data$sensor)*1000)/10
        
        print(paste0('Flagging identified ', n, ' (', perc, '%) outliers.'))
    }
    data
}

#### Bin Data by pressure and time
bin.pt = function(data, dp = 1, secs = 10) {
    i = 1
    while (i < nrow(data$sensor)) {
        l = which(abs(data$sensor$p - data$sensor$p[i]) + abs(data$sensor$time - data$sensor$time[i]) / secs < dp / 2 &
                 data$sensor$p.FLAG == 1)
        temp = apply(data$sensor[l,], 2, function(x) {mean(x, na.rm = TRUE)})
        
        if (length(l) > 1) {
            data$sensor[i,] = temp
            l = l[l != i]
            
            data$sensor = data$sensor[-l,]
        }
        i = i + 1
    }
    
    data
}

#### Bin Data by pressure
bin.p = function(data, dp = 1) {
    data$sensor = data$sensor[which(data$sensor$p.FLAG == 1),]
    data$sensor$p = floor((data$sensor$p + 0.5) / dp)
    data$sensor$n.bin = 1
    
    i = 1
    while (i < nrow(data$sensor)) {
        delta.p = abs(diff(data$sensor$p[i:nrow(data$sensor)]))  # i refenced
        
        ## Find entries to average across
        l = which(cumsum(delta.p) == 0)
        l = c(i, l + i)
        
        temp = as.numeric(apply(data$sensor[l,], 2, function(x) {median(x, na.rm = TRUE)}))
        
        ## Update values and some housekepping
        if (length(l) > 1 & length(temp) == ncol(data$sensor)) { ## Need to update values
            data$sensor[i,] = temp
            data$sensor$n.bin[i] = length(l)
            
            l = l[l != i]
            data$sensor = data$sensor[-l,] ## Remove other values
            delta.p = delta.p[-l]
        }
        i = i + 1
    }
    ## Finalize and return averaged data
    data$sensor$p = data$sensor$p * dp
    data
}

In [ ]:
check.bin = function(data, l = c(0000:135000)) {
    plot(data$sensor$time[l], data$sensor$p[l], type='l', ylab='Pressure', xlab='Time', ylim=c(100,0), yaxs='i')
    points(bin.data$sensor$time, bin.data$sensor$p, pch=4, cex=1.5, col='blue')
}

In [ ]:
preprocess.files = function(input.dir, files, verbose = FALSE) {
    for (f in 1:length(files)) {
        if (verbose) {
            print(paste0('Starting file: ', files[f], ' (', f, ' of ', length(files), ')'))
        }
        
        data = get.data(input.dir, files[f])
        data = flag.p.outliers(data)
        n.before = nrow(data$sensor)
        
        if (verbose) {print('Binning the data.')}
        data = bin.p(data)
        
        if (verbose) {print('Checkpoint Saved.')}
        if (verbose) {print('')}
        
        save(data, file = paste0('RStates/SeaSoar/Seasoar2-binned-', files[f], '.rdata'))
        
        if (verbose) {
            print(paste0('Number of Sensor Records before binning: ', n.before))
            print(paste0('Number of Sensor Records after binning: ', nrow(data$sensor)))
            print(paste0('(', floor(nrow(data$sensor)/n.before * 1000) / 10, '%)'))
        }
        print('')
    }
}

load.rstate = function(input.dir, files, verbose = TRUE) {
    compiled.data = NULL
    
    for (f in 1:length(files)) {
        if (verbose) {print(paste0('Starting file: ', files[f], ' (', f, ' of ', length(files), ')'))}
        load(file = paste0('RStates/SeaSoar/Seasoar2-binned-', files[f], '.rdata'))
        
        if (is.null(compiled.data)) {
            compiled.data = data
        } else {
            compiled.data$sensor = rbind(compiled.data$sensor, data$sensor)
            compiled.data$eng = rbind(compiled.data$eng, data$eng)
        }
        print(nrow(data$sensor))
    }
    compiled.data
}

In [ ]:
#preprocess.files(input.dir, files, verbose = TRUE)

In [ ]:
data = load.rstate(input.dir, files)

In [ ]:
conv.time = function(x, tz = 'UTC') {
    as.POSIXct(x, origin="1970-01-01", tz = tz)
}

add.times = function(data, tz = 'UTC') {
    data$sensor$time.real = as.POSIXct(data$sensor$time, origin="1970-01-01", tz = tz)
    data$eng$time.real = as.POSIXct(data$eng$time, origin="1970-01-01", tz = tz)
    
    data
}

In [ ]:
for (i in 1:ncol(data$sensor)) {
    data$sensor[,i] = as.numeric(data$sensor[,i])
}
str(data)

In [ ]:
data = add.times(data)

In [ ]:
add.direction = function(data, p.min = 8, p.max = 250) {
    ## value codes
    ##   -1 = downcast
    ##   +1 = upcast
    ##    0 = ambiguous/outside bounds
    
    data$sensor$direction = 0
    dp = diff(data$sensor$p)
    
    l = which(dp < 0) # getting shallower
    data$sensor$direction[l+1] = 1
    
    l = which(dp > 0) # getting depper
    data$sensor$direction[l+1] = -1
    
    data
}

In [ ]:
data = add.direction(data)

In [ ]:
get.mld = function(data, rho = 0.1) {
    mld = data.frame(time = 0, lat = 0, lon = 0, mld = 0, stringsAsFactors = FALSE)
    
    ## Trim data to between 5db and 100db
    data$sensor = data$sensor[data$sensor$p > 5 & data$sensor$p < 100,]
    
    ## Find all the 10meter points on the downcast
    l = which(data$sensor$p == 10 & data$sensor$direction == -1)
    
    ## l should only include entries from different casts: here 10 minutes apart.
    dt = as.numeric(difftime(data$sensor$time.real[l], data$sensor$time.real[l[1]], units='mins'))
    l = l[c(1, which(diff(dt) > 10))]
    
    print(length(l))
    
    ## calculate mld and add to dataframe
    for (i in 1:length(l)) {
        t.10 = data$sensor$time.real[l[i]]
        rho.10 = data$sensor$sigma2[l[i]]
        
        ## define cast as within 3 minutes, downcast, pressure > 10, and afterwards
        l.time = which(as.numeric(difftime(data$sensor$time.real, t.10, unit='mins')) < 5 &
                       data$sensor$direction == -1 & data$sensor$p > 10 &
                       as.numeric(difftime(data$sensor$time.real, t.10, unit='mins')) > 0)
        
        if (length(l.time) > 15) {
            l.mld = min(which(data$sensor$sigma2[l.time] > rho.10 + rho))
            t.mld = data$sensor$time.real[l.time[l.mld]]

            eng = which.min(as.numeric(difftime(data$eng$time.real, t.10, units = 'mins'))^2)
            mld = rbind(mld, c(t.mld, data$eng$lat[eng], data$eng$lon[eng], data$sensor$p[l.time[l.mld]]))
        }
    }
    
    mld = mld[-1,]
    mld$time = conv.time(mld$time)
    mld
}

In [ ]:
mld = get.mld(data, 0.1)
mld2 = get.mld(data, 0.2)

In [ ]:
#plot(data$sensor$time.real, data$sensor$p, cex=0.1, pch=20, col=get.qual.pal(3)[data$sensor$direction+2],
#     ylim=c(300,0), yaxs='i', ylab='Depth', xlab='')
plot(data$sensor$time.real[c(1, nrow(data$sensor))], data$sensor$p[c(1, nrow(data$sensor))], cex=0.1, pch=20, col='white',
     ylim=c(100,0), yaxs='i', ylab='Depth', xlab='')

points(mld$time, mld$mld, pch=15, col='dark red')
points(mld2$time, mld2$mld, pch=16, col='dark blue', cex=0.5)
lines(c(min(mld$time), max(mld$time)), c(10,10), lty=2)

In [ ]:
plot.map(lon = mld$lon, lat = mld$lat, main = 'SeaSoar MLDs', col = make.div.pal(mld$mld, 100))

In [ ]:
seasoar2 = mld
save(seasoar2, file = 'RStates/SeaSoar2.MLD (0.1).rdata')

In [ ]:
save(data, file='RStates/SeaSoar2.final.rdata')

---
# Review

In [ ]:
plot.mld = function(mld, i, window = 3) {
    l = which(as.numeric(difftime(data$sensor$time.real, mld$time[i], units = 'mins')) < window &
              as.numeric(difftime(data$sensor$time.real, mld$time[i], units = 'mins')) >= -window &
             data$sensor$direction == -1)
    
    plot(data$sensor$sigma2[l], data$sensor$p[l], ylim=c(100,0), xlim=c(1023,1027), yaxs='i',
         pch=20, col='#00000050', cex=1, main=paste0(i, ' - ', length(l)), ylab='Depth', xlab='Density')
    
    lines(c(0,10000), rep(mld$mld[i], 2), lty=2, col='red')
    #lines(c(0,10000), rep(mld2$mld[i], 2), lty=1, col='red')
    lines(c(0,10000), rep(10, 2), lty=2, col='blue')
    
    text(1023, 80, mld$mld[i])
}

In [ ]:
pdf('Output/SeaSoar2 - MLD Review (0.2).pdf')

par(mfrow=c(2,2))
for(i in 1:nrow(mld2)) {
    plot.mld(mld2, i)
}

dev.off()